In [1]:
!pip install pandas

In [2]:
!pip install scipy

In [3]:
import pandas as pd
from scipy.stats import pearsonr, ttest_ind

# Load the datasets
step_data = pd.read_csv('stepcountck.csv')  # Step count data
weather_data = pd.read_csv('sabanciwdata_converted.csv')  # Weather data

# Convert timestamps to datetime format
step_data['startDate'] = pd.to_datetime(step_data['startDate'])
weather_data['Time'] = pd.to_datetime(weather_data['Time'])

# Extract the date for daily aggregation
step_data['Date'] = step_data['startDate'].dt.date
weather_data['Date'] = weather_data['Time'].dt.date

# Aggregate step counts to daily totals
daily_steps = step_data.groupby('Date').agg(DailyStepCount=('value', 'sum')).reset_index()

# Aggregate weather data to daily averages
daily_weather = weather_data.groupby('Date').agg(
    DailyTemp=('Temp', 'mean'),
    DailyWindSpeed=('Wspd', 'mean')
).reset_index()

# Merge the datasets on 'Date'
merged_data = pd.merge(daily_steps, daily_weather, on='Date', how='inner')

# Calculate correlations and p-values
cor_temp, p_value_temp = pearsonr(merged_data['DailyTemp'], merged_data['DailyStepCount'])
cor_wind, p_value_wind = pearsonr(merged_data['DailyWindSpeed'], merged_data['DailyStepCount'])

# Results
correlations = {
    'DailyTemp': {'Correlation': cor_temp, 'P-Value': p_value_temp},
    'DailyWindSpeed': {'Correlation': cor_wind, 'P-Value': p_value_wind}
}
correlations

{'DailyTemp': {'Correlation': np.float64(-0.02216675702351406),
  'P-Value': np.float64(0.7722140273597353)},
 'DailyWindSpeed': {'Correlation': np.float64(-0.07531751046965784),
  'P-Value': np.float64(0.3246903325381117)}}